# Identify & Track Marine Heatwaves on _Unstructured Grid_ using `spot_the_blOb`

## Processing Steps:
1. Fill spatial holes in the binary data, using `dask_image.ndmorph` -- up to `R_fill` cells in radius.
2. Fill gaps in time -- permitting up to `T_fill` missing time slices, while keeping the same blob ID.
3. Filter out small objects -- area less than the bottom `area_filter_quartile` of the size distribution of objects.
4. Identify objects in the binary data, using `dask_image.ndmeasure`.
5. Connect objects across time, applying the following criteria for splitting, merging, and persistence:
    - Connected Blobs must overlap by at least fraction `overlap_threshold` of the smaller blob.
    - Merged Blobs retain their original ID, but partition the child blob based on the parent of the _nearest-neighbour_ cell. 
6. Cluster and reduce the final object ID graph using `scipy.sparse.csgraph.connected_components`.
7. Map the tracked objects into ID-time space for convenient analysis.

N.B.: Exploits parallelised `dask` operations with optimised chunking using `flox` for memory efficiency and speed \
N.N.B.: This example using 40 years of _daily_ outputs at 5km resolution on an Unstructured Grid (15 million cells) using 32 cores takes 
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [ ]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import spot_the_blOb as blob
import spot_the_blOb.helper as hpc

In [ ]:
# Start Dask Cluster
client = hpc.StartLocalCluster(n_workers=32, n_threads=1)

2024-12-10 19:33:01,653 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:41173' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('transpose-abc76ff9aa40c83a8fe04e81f638a5f1', 28, 0), ('transpose-1ce20af2cae8681225cd60a2d38315da', 28, 0), ('transpose-24b3232b2164dbdc36663306593b64d1', 28, 0), ('_trim-151b707e02d9935f1acff686b570b313', 28, 0), ('transpose-9751b33b295dc47eac5b025839d4a25f', 28, 0)} (stimulus_id='handle-worker-cleanup-1733855581.6523628')
2024-12-10 19:33:01,656 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:46429' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('transpose-abc76ff9aa40c83a8fe04e81f638a5f1', 16, 0), ('transpose-24b3232b2164dbdc36663306593b64d1', 16, 0), ('transpose-1ce20af2cae8681225cd60a2d38315da', 16, 0), ('transpose-9751b33b295dc47eac5b025839d4a25f', 16, 0), ('_trim-151b707e02d9935f1acff686b570b313', 16, 0)} (stimulus_id='handle-

In [ ]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'extreme_events_binary_unstruct.zarr'
chunk_size = {'time': 4, 'lat': -1, 'lon': -1}
ds = xr.open_zarr(str(file_name), chunks=chunk_size).isel(time=slice(0,128))

In [ ]:
# Tracking Parameters

drop_area_quartile = 0.5  # Remove the smallest 50% of the identified blobs
hole_filling_radius = 32   # Fill small holes with radius < 8 elements
time_gap_fill = 2         # Allow gaps of 2 days and still continue the blob tracking with the same ID
allow_merging = True      # Allow blobs to split/merge. Keeps track of merge events & unique IDs.
overlap_threshold = 0.5   # Overlap threshold for merging blobs. If overlap < threshold, blobs keep independent IDs.
nn_partitioning = True    # Use new NN method to partition merged children blobs. If False, reverts to old method of Di Sun et al. 2023...

In [ ]:
# SpOt & Track the Blobs & Merger Events

tracker = blob.Spotter(ds.extreme_events, ds.mask, R_fill=hole_filling_radius, T_fill = time_gap_fill, area_filter_quartile=drop_area_quartile, 
                       allow_merging=allow_merging, overlap_threshold=overlap_threshold, nn_partitioning=nn_partitioning, 
                       xdim='ncells',               # Need to tell spot_the_blOb the new Unstructured dimension
                       unstructured_grid=True,      # Use Unstructured Grid
                       neighbours=ds.neighbours)    # Connectivity array for the Unstructured Grid
# blobs = tracker.run(return_merges=False)

# blobs

In [ ]:
data_bin_filled = tracker.fill_holes().persist()
data_bin_filled

In [ ]:
data_bin_gap_filled = tracker.fill_time_gaps(data_bin_filled).persist()
data_bin_gap_filled

In [ ]:
data_bin_filtered, area_threshold, blob_areas, N_blobs_unfiltered = tracker.filter_small_blobs(data_bin_gap_filled)

In [ ]:
data_bin_filtered = data_bin_filtered.persist()

In [ ]:
# Save Tracked Blobs to `zarr` for more efficient parallel I/O

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'MHWs_tracked_unstruct.zarr'
blobs.to_zarr(file_name, mode='w')